"""Documentação da Análise de Classes Gramaticais no MacMorpho"""

# Introdução
Este notebook descreve a tarefa, metodologia e os resultados obtidos na análise de classes gramaticais utilizando um modelo de classificação baseado em Transformers. O objetivo principal é avaliar a precisão de diferentes classes gramaticais em uma tarefa de etiquetagem POS (Part-of-Speech).

# Descrição da Tarefa
A tarefa consiste em treinar e avaliar um modelo para etiquetar palavras em sentenças com suas respectivas classes gramaticais. O dataset utilizado é o MacMorpho, amplamente utilizado para o processamento de linguagem natural em português. A análise inclui:

1. Treinamento de um modelo baseado no BERT pré-treinado ("neuralmind/bert-base-portuguese-cased").
2. Avaliação do modelo no conjunto de teste.
3. Geração de métricas por classe gramatical.

# Metodologia

## Dados

Os dados foram obtidos do dataset MacMorpho, e foram divididos em:
- Conjunto de treinamento.
- Conjunto de teste.

Cada palavra no dataset é associada a uma tag que representa sua classe gramatical. Essas classes incluem substantivos (N), verbos (V), adjetivos (ADJ), entre outros.

### Pré-processamento
Os dados foram pré-processados para:
- Tokenizar as palavras utilizando o tokenizer do BERT.
- Mapear tags para índices e vice-versa.
- Gerar entradas no formato compatível com modelos da biblioteca Transformers.

## Modelo

Utilizou-se o modelo "neuralmind/bert-base-portuguese-cased", adaptado para classificação de tokens com o número de labels correspondente às classes gramaticais do MacMorpho. O modelo foi treinado com:
- 3 épocas.
- Batch size de 8.
- Taxa de aprendizado inicial de 5e-5.

### Métricas de Avaliação
As métricas utilizadas para avaliar o modelo incluem:
- Precisão (Precision)
- Revocação (Recall)
- F1-Score

Além disso, foram calculadas métricas por classe gramatical.

# Resultados

## Métricas Gerais
As métricas gerais obtidas foram:
```python
metrics = trainer.evaluate()
print("Precisão geral:", metrics["precision"])
print("Revocação geral:", metrics["recall"])
print("F1 geral:", metrics["f1"])
```

## Análise por Classe Gramatical
Para avaliar o desempenho por classe, foi gerado um relatório detalhado:

```python
def evaluate_model_per_class(model, tokenizer, X_test, y_test, label_classes):
    # Implementação detalhada no código abaixo

report = evaluate_model_per_class(model, tokenizer, X_test, y_test, test_data.unique_tags)
print(report)
```

### Classes com Maior Precisão
Com base na análise, as classes gramaticais com maior precisão foram:
- Substantivos (N)
- Verbos (V)

### Classes com Menor Precisão
As classes com menor precisão incluíram:
- Advérbios (ADV)
- Preposições (PRP)

# Conclusão
O modelo baseado em BERT demonstrou bom desempenho geral, mas apresentou variação significativa entre as classes gramaticais. Classes mais frequentes no dataset, como substantivos e verbos, tendem a obter maior precisão. Por outro lado, classes menos frequentes, como advérbios, apresentam menor precisão, indicando a necessidade de mais dados ou ajustes para essas classes específicas.

# Próximos Passos
- Experimentar diferentes arquiteturas de modelos.
- Realizar fine-tuning com mais épocas ou técnicas de data augmentation.
- Investigar as classes com baixa precisão para entender as causas (por exemplo, ambiguidade linguística ou escassez de exemplos no dataset).

# Data

In [1]:
import os

main_file_path = "data/macmorpho-v3.tgz"
test_filepath = "data/macmorpho-test.txt"
train_filepath = "data/macmorpho-train.txt"

if not os.path.exists(main_file_path):
    os.makedirs("data", exist_ok=True)
    os.system("wget -P data/ http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz")
    os.system("tar -xvzf data/macmorpho-v3.tgz -C data/")
    print("Dataset imported\n\n")
else:
    print("Dataset was already imported\n\n")

os.system("pip install 'accelerate>=0.26.0'")
os.system("pip install transformers seqeval")

print("Pip done\n\n")

import os
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

device = 'cuda' if torch.cuda.is_available() else 'cpu'

--2025-01-19 19:42:15--  http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz
Resolving nilc.icmc.usp.br (nilc.icmc.usp.br)... 143.107.183.225
Connecting to nilc.icmc.usp.br (nilc.icmc.usp.br)|143.107.183.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2463485 (2.3M) [application/x-gzip]
Saving to: ‘macmorpho-v3.tgz’

macmorpho-v3.tgz    100%[===================>]   2.35M  1.90MB/s    in 1.2s    

2025-01-19 19:42:17 (1.90 MB/s) - ‘macmorpho-v3.tgz’ saved [2463485/2463485]

macmorpho-dev.txt
macmorpho-test.txt
macmorpho-train.txt


# Get Data

In [ ]:
class MacMorphoData:
    def __init__(self, filepath, tokenizer, max_length=50):
        self.filepath = filepath
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Read lines
        with open(self.filepath, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')

        # Each line -> list of "word_POS" strings
        items = [line.strip().split() for line in lines if line.strip()]

        # Turn each sentence into a list of (word, tag) tuples
        self.wt = [
            [tuple(unit.split('_')) for unit in sentence]
            for sentence in items
        ]

        # Flatten all (word, tag) pairs across the entire file
        pairs = [(word, tag) for sentence in self.wt for (word, tag) in sentence]

        # Separate words and tags
        self.words, self.tags = zip(*pairs)  # 'words' and 'tags' are tuples now
        self.words = list(self.words)
        self.tags = list(self.tags)

        # Build unique tags
        self.unique_tags = sorted(list(set(self.tags)))

        # Tag <-> index maps
        self.tag2idx = {tag: idx for idx, tag in enumerate(self.unique_tags)}
        self.idx2tag = {idx: tag for tag, idx in self.tag2idx.items()}

        # Now let's preprocess (tokenize) all sentences
        self.processed_data = self._process_all_sentences()
        # self.processed_data is a list of dicts:
        #   [{'input_ids':..., 'attention_mask':..., 'labels':...}, ...]

    def _process_sentence(self, sentence):
        """
        sentence: list of (word, tag) pairs, e.g. [("Jersei","N"), ("atinge","V"), ("média","N")]
        returns a dict with input_ids, attention_mask, labels
        """
        words = [w for (w, t) in sentence]
        tags = [t for (w, t) in sentence]

        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            padding='max_length',    # or True, or 'longest'
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'      # return PyTorch tensors
        )

        input_ids = encoding['input_ids'][0]        # shape (seq_len,)
        attention_mask = encoding['attention_mask'][0]  # shape (seq_len,)

        # Use word_ids() to map back to original words
        word_ids = encoding.word_ids(batch_index=0)

        # Build label_ids aligned with subwords
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                # Special tokens [CLS], [SEP], or padded subwords
                label_ids.append(-100)
            else:
                # Use the word_id to find the original tag
                original_tag = tags[word_id]
                label_id = self.tag2idx[original_tag]
                label_ids.append(label_id)

        # Return a dictionary
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label_ids)
        }

    def _process_all_sentences(self):
        """
        Helper to process/tokenize all sentences in self.wt
        """
        processed = []
        for sentence in self.wt:
            processed_example = self._process_sentence(sentence)
            processed.append(processed_example)
        return processed

    def __len__(self):
        """
        Number of sentences (before tokenization).
        """
        return len(self.wt)

import torch
from torch.utils.data import Dataset

class POSDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings  # list of dicts

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return {k: v for k, v in self.encodings[idx].items()}

# Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

train_data = MacMorphoData(train_filepath, tokenizer, max_length=50)
test_data = MacMorphoData(test_filepath, tokenizer, max_length=50)

from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

train_dataset = POSDataset(train_data.processed_data)
eval_dataset = POSDataset(test_data.processed_data)

num_labels = len(train_data.unique_tags)
model = AutoModelForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=num_labels
)

# Make model aware of label mapping
model.config.id2label = train_data.idx2tag
model.config.label2id = train_data.tag2idx

training_args = TrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=5e-5
)
def align_predictions(predictions, label_ids):
    preds = predictions.argmax(-1)
    batch_size, seq_len = preds.shape
    out_preds, out_labels = [], []

    for i in range(batch_size):
        pred_i = []
        label_i = []
        for j in range(seq_len):
            if label_ids[i][j] != -100:
                pred_i.append(model.config.id2label[preds[i][j].item()])
                label_i.append(model.config.id2label[label_ids[i][j].item()])
        out_preds.append(pred_i)
        out_labels.append(label_i)
    return out_preds, out_labels

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.from_numpy(logits)
    label_ids = torch.from_numpy(labels)

    pred_tags, true_tags = align_predictions(predictions, label_ids)
    precision = precision_score(true_tags, pred_tags)
    recall = recall_score(true_tags, pred_tags)
    f1 = f1_score(true_tags, pred_tags)
    return {"precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Train and Evaluate Model

In [ ]:

####################
# 5) Train
####################
print("Starting training\n\n")
trainer.train()
print("Training ended\n\n")


####################
# 5) Save
####################
trainer.save_model("results")


####################
# 6) Evaluate
####################

"""###Evaluate overall"""
print("Starting evaluation\n\n")
metrics = trainer.evaluate()
print("Overall Evaluation ended\n\n")
print("Overall Evaluation:", metrics)

from tqdm import tqdm

from sklearn.metrics import classification_report as sklearn_classification_report


"""###Evaluate per class"""
def evaluate_model_per_class(model, tokenizer, X_test, y_test, label_classes):
    model.eval()  # Set model to evaluation mode

    y_pred = []
    y_true = []

    with torch.no_grad():
        for sentence, true_tags in tqdm(zip(X_test, y_test), total=len(X_test), desc="Evaluating", unit="sentence"):
            # Tokenize input
            inputs = tokenizer(sentence, return_tensors="pt", is_split_into_words=True, truncation=True)
            outputs = model(**inputs)

            # Get predictions (logits -> argmax -> predicted labels)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Map predictions back to the original tags
            word_ids = inputs.word_ids(batch_index=0)
            predicted_tags = []
            for idx, word_id in enumerate(word_ids):
                if word_id is None:  # Skip special tokens
                    continue
                if word_id != word_ids[idx - 1]:  # Only consider the first subword
                    predicted_tags.append(model.config.id2label[predictions[0, idx].item()])

            # Ensure the prediction length matches the true tag length
            if len(predicted_tags) == len(true_tags):
                y_pred.append(predicted_tags)
                y_true.append(true_tags)
            else:
                print(f"Length mismatch: predicted {len(predicted_tags)}, true {len(true_tags)}")

    # Flatten the lists for classification report
    y_true_flat = [tag for sentence in y_true for tag in sentence]
    y_pred_flat = [tag for sentence in y_pred for tag in sentence]

    # Generate and print the classification report
    report = sklearn_classification_report(y_true_flat, y_pred_flat, labels=label_classes, zero_division=0)
    print(report)

    return report


X_test = [[word for word, tag in sentence] for sentence in test_data.wt]
y_test = [[tag for word, tag in sentence] for sentence in test_data.wt]

metrics_per_class = evaluate_model_per_class(model, tokenizer, X_test, y_test, test_data.unique_tags)

In [ ]:
# Save evaluation 

import pickle

pickle_file = "results/evaluation_metrics.pkl"
with open(pickle_file, "wb") as f:
    pickle.dump(metrics, f)
print(f"\n\nEvaluation saved to {pickle_file}\n\n")

per_class_pickle_file = "results/evaluation_metrics_per_class.pkl"
with open(per_class_pickle_file, "wb") as f:
    pickle.dump(metrics_per_class, f)
print(f"\n\nEvaluation saved to {per_class_pickle_file}\n\n")

# Show results

In [3]:
import pickle

pickle_file = "results/evaluation_metrics.pkl"
with open(pickle_file, "rb") as f:
    data_metrics_1 = pickle.load(f)
print(f"\n\nOverall Evaluation\n{data_metrics_1}\n\n")

per_class_pickle_file = "results/evaluation_metrics_per_class.pkl"
with open(per_class_pickle_file, "rb") as f:
    data_metrics_2 = pickle.load(f)
print(f"\n\nEvaluation per class\n{data_metrics_2}\n\n")



Overall Evaluation
{'eval_loss': 0.10592348128557205, 'eval_precision': 0.9766553452805465, 'eval_recall': 0.9778856948904728, 'eval_f1': 0.9772701328436634, 'eval_runtime': 64.4366, 'eval_samples_per_second': 154.989, 'eval_steps_per_second': 19.383, 'epoch': 3.0}




Evaluation per class
              precision    recall  f1-score   support

         ADJ       0.96      0.96      0.96      8554
         ADV       0.93      0.93      0.93      5446
      ADV-KS       0.84      0.86      0.85       230
         ART       0.99      0.99      0.99     12580
         CUR       1.00      1.00      1.00       296
          IN       0.54      0.77      0.63        98
          KC       0.99      0.98      0.98      4531
          KS       0.94      0.93      0.93      2538
           N       0.98      0.98      0.98     36542
       NPROP       0.98      0.98      0.98     15936
         NUM       0.97      0.96      0.97      2541
         PCP       0.98      0.97      0.97      3640
    